In [17]:
import torch
from IPython.display import Audio, display
from moshi.models.loaders import CheckpointInfo
from moshi.models.tts import DEFAULT_DSM_TTS_VOICE_REPO, TTSModel


voice = "/content/REC-20260109013624.wav"

checkpoint_info = CheckpointInfo.from_hf_repo('kyutai/tts-0.75b-en-public')
tts_model = TTSModel.from_checkpoint_info(
    checkpoint_info, n_q=16, temp=0.6, cfg_coef=3, device=torch.device("cuda")
)
# `voice` could also be a local wav file.
voice_path = tts_model.get_voice_path(voice)
prefix = tts_model.get_prefix(voice_path)

In [18]:
text = ""
entries = tts_model.prepare_script([text], padding_between=1)
print("Generating audio...")
pcms = []
def _on_frame(frame):
    print("Step", len(pcms), end="\r")
    if (frame[:, 1:] != -1).all():
        pcm = tts_model.mimi.decode(frame[:, 1:, :]).cpu()
        pcms.append(pcm.clip(-1, 1))

# You could also generate multiple audios at once by extending the following lists.
all_entries = [entries]
prefixes = [prefix]
with tts_model.mimi.streaming(len(all_entries)):
    result = tts_model.generate(all_entries, [], on_frame=_on_frame, prefixes=prefixes)

print("Done generating.")
audios = torch.cat(pcms, dim=-1)

for audio, prefix in zip(audios, prefixes):
    # We need to skip the audio prefix.
    skip = int((tts_model.mimi.sample_rate * prefix.shape[-1]) / tts_model.mimi.frame_rate)
    audio = audio[..., skip:]
    # Now do something with this audio!

Generating audio...
Done generating.


In [19]:
display(Audio(audio, rate=tts_model.mimi.sample_rate))

In [8]:
!cd VibeVoice
!python demo/inference_from_file.py \
--model_path "tarun7r/vibevoice-hindi-1.5B" \
--txt_path "text.txt" \
--speaker_names "REC-20260109012725.wav" \
--cfg_scale 1.3


python3: can't open file '/content/demo/inference_from_file.py': [Errno 2] No such file or directory
